# [DA_LAB] 2. 연관규칙분석: 특허데이터 분석

## Contents
* 데이터 불러오기
* 특허등록번호: IPC 트랜잭션 구조의 딕셔너리 생성
* 트랜잭션 데이터
* One-hot Array
* 빈발집합 생성
* 연관규칙 분석

## Used Library
* pandas
* mlxtend
* pymysql

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pymysql

### 데이터 불러오기

In [4]:
conn = pymysql.connect(host="chamaeleon.konkuk.ac.kr", user="class_da", password="dataanalytics", db="tp_class_da", charset="utf8")
cursor = conn.cursor()
sql = """select paaa.특허등록번호 as reg_num, paaa.IPC4 as ipc4 from
            (select paa.특허등록번호, left(ipc.보유IPC전체코드,4) as IPC4 from 
                (SELECT * FROM tp_class_da.us_patent pa order by pa.특허등록일자 desc limit 1000) paa 
                    inner join tp_class_da.ipc on paa.특허등록번호=ipc.특허등록번호) paaa 
            group by paaa.특허등록번호, paaa.IPC4;"""
cursor.execute(sql)
db=cursor.fetchall()

### 특허등록번호: IPC 트랜잭션 구조의 딕셔너리 생성

In [5]:
ipc_dic={}
ipc_transaction=[]

In [6]:
for reg_num, ipc in db:
    if ipc_dic.get(reg_num):
        ipc_dic[reg_num].append(ipc)
    else:
        ipc_dic[reg_num]=[ipc]
        

In [7]:
list(ipc_dic.items())[:10]

[('9848521', ['A01B', 'B07B', 'E02F']),
 ('9848522', ['A01B', 'G01L']),
 ('9848523', ['A01B', 'A01C', 'G01L']),
 ('9848524', ['A01B', 'A01C']),
 ('9848525', ['A01C', 'B65G']),
 ('9848526', ['A01B', 'A01C']),
 ('9848527', ['A01B', 'A01C']),
 ('9848528', ['A01C', 'G01S']),
 ('9848529', ['A01D']),
 ('9848530', ['A01D', 'A01F'])]

### 트랜잭션 데이터

In [8]:
ipc_transaction=list(ipc_dic.values())

In [11]:
ipc_transaction[:10]

[['A01B', 'B07B', 'E02F'],
 ['A01B', 'G01L'],
 ['A01B', 'A01C', 'G01L'],
 ['A01B', 'A01C'],
 ['A01C', 'B65G'],
 ['A01B', 'A01C'],
 ['A01B', 'A01C'],
 ['A01C', 'G01S'],
 ['A01D'],
 ['A01D', 'A01F']]

### one-hot array로 변환

In [12]:
te=TransactionEncoder()
te_array=te.fit(ipc_transaction).transform(ipc_transaction)
te_array.astype("int")

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
df=pd.DataFrame(te_array, columns=te.columns_)

In [14]:
df[:10]

,A01B,A01C,A01D,A01F,A01G,A01H,A01J,A01K,A01M,A01N,...,H02N,H04B,H04L,H04N,H04Q,H04R,H04W,H05B,H05H,H05K
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### 빈발집합 생성(최소 Support 값 설정)

In [15]:
frequent_itemsets=apriori(df, min_support=0.005, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[:10]

,support,itemsets,length
0,0.008,(A01B),1
1,0.006,(A01C),1
2,0.009,(A01D),1
3,0.011,(A01G),1
4,0.029,(A01H),1
5,0.016,(A01K),1
6,0.008,(A01M),1
7,0.016,(A01N),1
8,0.005,(A23C),1
9,0.005,(A23K),1


### 연관규칙 생성

In [16]:
rules=association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
rules[:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(A01H),(C12N),0.029,0.041,0.024,0.827586,20.185029,0.022811,5.562200
1,(C12N),(A01H),0.041,0.029,0.024,0.585366,20.185029,0.022811,2.341824
2,(A01N),(A61K),0.016,0.153,0.010,0.625000,4.084967,0.007552,2.258667
3,(A24F),(H05B),0.024,0.011,0.006,0.250000,22.727273,0.005736,1.318667
4,(H05B),(A24F),0.011,0.024,0.006,0.545455,22.727273,0.005736,2.147200
5,(B65D),(A47J),0.023,0.017,0.005,0.217391,12.787724,0.004609,1.256056
6,(A47J),(B65D),0.017,0.023,0.005,0.294118,12.787724,0.004609,1.384083
7,(A61C),(A61B),0.013,0.245,0.005,0.384615,1.569859,0.001815,1.226875
8,(A61F),(A61B),0.109,0.245,0.047,0.431193,1.759970,0.020295,1.327339
9,(A61B),(A61F),0.245,0.109,0.047,0.191837,1.759970,0.020295,1.102500


In [17]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules[:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(A01H),(C12N),0.029,0.041,0.024,0.827586,20.185029,0.022811,5.562200,1
1,(C12N),(A01H),0.041,0.029,0.024,0.585366,20.185029,0.022811,2.341824,1
2,(A01N),(A61K),0.016,0.153,0.010,0.625000,4.084967,0.007552,2.258667,1
3,(A24F),(H05B),0.024,0.011,0.006,0.250000,22.727273,0.005736,1.318667,1
4,(H05B),(A24F),0.011,0.024,0.006,0.545455,22.727273,0.005736,2.147200,1
5,(B65D),(A47J),0.023,0.017,0.005,0.217391,12.787724,0.004609,1.256056,1
6,(A47J),(B65D),0.017,0.023,0.005,0.294118,12.787724,0.004609,1.384083,1
7,(A61C),(A61B),0.013,0.245,0.005,0.384615,1.569859,0.001815,1.226875,1
8,(A61F),(A61B),0.109,0.245,0.047,0.431193,1.759970,0.020295,1.327339,1
9,(A61B),(A61F),0.245,0.109,0.047,0.191837,1.759970,0.020295,1.102500,1


In [19]:
rules.sort_values(by=['support'], ascending=False)[:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
8,(A61F),(A61B),0.109,0.245,0.047,0.431193,1.759970,0.020295,1.327339,1
9,(A61B),(A61F),0.245,0.109,0.047,0.191837,1.759970,0.020295,1.102500,1
11,(A61M),(A61B),0.096,0.245,0.029,0.302083,1.232993,0.005480,1.081791,1
12,(A61B),(A61M),0.245,0.096,0.029,0.118367,1.232993,0.005480,1.025370,1
0,(A01H),(C12N),0.029,0.041,0.024,0.827586,20.185029,0.022811,5.562200,1
1,(C12N),(A01H),0.041,0.029,0.024,0.585366,20.185029,0.022811,2.341824,1
13,(A61N),(A61B),0.051,0.245,0.019,0.372549,1.520608,0.006505,1.203281,1
31,(A61K),(C07K),0.153,0.019,0.018,0.117647,6.191950,0.015093,1.111800,1
30,(C07K),(A61K),0.019,0.153,0.018,0.947368,6.191950,0.015093,16.093000,1
27,(A61K),(A61L),0.153,0.040,0.017,0.111111,2.777778,0.010880,1.080000,1
